### 复现课堂代码

In [1]:
import numpy as np
import pandas as pd
import jieba
import random

In [2]:
simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>  蓝色的 | 好看的 | 小小的
"""

In [3]:
simple_grammar

'\nsentence => noun_phrase verb_phrase\nnoun_phrase => Article Adj* noun\nAdj* => null | Adj Adj*\nverb_phrase => verb noun_phrase\nArticle =>  一个 | 这个\nnoun =>   女人 |  篮球 | 桌子 | 小猫\nverb => 看着   |  坐在 |  听着 | 看见\nAdj =>  蓝色的 | 好看的 | 小小的\n'

In [4]:
def adj(): return np.random.choice(list(map(lambda s: s.strip(), '蓝色的 | 好看的 | 小小的'.split('|'))))

In [5]:
adj()

'蓝色的'

In [6]:
def adj_star(): return np.random.choice([lambda : '', lambda : adj()+adj_star()])()

In [7]:
adj_star()

'好看的好看的'

In [8]:
list(map(lambda s: s.strip(), '蓝色的 | 好看的 | 小小的'.split('|')))

['蓝色的', '好看的', '小小的']

In [9]:
adj_grammar = """
Adj* => null | Adj Adj*
Adj =>  蓝色的 | 好看的 | 小小的
"""

In [10]:
def create_grammar(grammar_str, split_='=>', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        k, v = line.split(split_)
        grammar[k.strip()] = [s.split() for s in v.split('|')]
    return grammar

In [11]:
create_grammar(adj_grammar)

{'Adj*': [['null'], ['Adj', 'Adj*']], 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [12]:
def generate(gram, target):
    if target not in gram: return target
    expanded = [generate(gram, t) for t in random.choice(gram[target])]
    return ''.join(s for s in expanded if s != 'null')

In [13]:
example_grammar = create_grammar(simple_grammar)
example_grammar

{'sentence': [['noun_phrase', 'verb_phrase']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'verb_phrase': [['verb', 'noun_phrase']],
 'Article': [['一个'], ['这个']],
 'noun': [['女人'], ['篮球'], ['桌子'], ['小猫']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见']],
 'Adj': [['蓝色的'], ['好看的'], ['小小的']]}

In [14]:
generate(example_grammar, 'sentence')

'一个小猫看见一个好看的小猫'

In [15]:
from collections import Counter

In [16]:
with open('article_9k.txt', 'r') as f:
    articles = list(map(lambda s: s.strip('\n'), f.readlines()))

In [17]:
articles[:2]

['此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MIUI8去年5月发布距今已有一年有余也是时候更新换代了当然关于MIUI9的确切信息我们还是等待官方消息',
 '骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台']

In [18]:
with_jieba_cut = Counter(jieba.cut(articles[1]))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xxx\AppData\Local\Temp\jieba.cache
Loading model cost 0.807 seconds.
Prefix dict has been built succesfully.


In [19]:
with_jieba_cut

Counter({'骁龙': 4,
         '835': 2,
         '作为': 2,
         '唯一': 1,
         '通过': 1,
         'Windows10': 3,
         '桌面': 3,
         '平台': 3,
         '认证': 1,
         '的': 8,
         'ARM': 1,
         '处理器': 1,
         '高通': 2,
         '强调': 1,
         '不会': 1,
         '因为': 1,
         '只': 1,
         '考虑': 1,
         '性能': 2,
         '而': 1,
         '去': 1,
         '屏蔽掉': 1,
         '小': 1,
         '核心': 1,
         '相反': 1,
         '他们': 1,
         '正': 1,
         '联手': 1,
         '微软': 2,
         '找到': 1,
         '一种': 1,
         '适合': 1,
         '兼顾': 1,
         '和': 1,
         '功耗': 1,
         '完美': 1,
         '方案': 2,
         '报道': 1,
         '称': 1,
         '已经': 1,
         '拿到': 1,
         '了': 2,
         '一些': 1,
         '新': 1,
         '源码': 1,
         '以便': 1,
         '更好': 1,
         '地': 1,
         '理解': 1,
         'biglittle': 1,
         '架构': 1,
         '资料': 1,
         '显示': 1,
         '一款': 1,
         '集成': 1,
   

In [20]:
articles[1]

'骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台'

In [21]:
def cut(string): return list(jieba.cut(string))

In [22]:
token = []
for sentence in articles[:10000]:
    token += cut(sentence)

In [23]:
words_count = Counter(token)

In [24]:
words_count.most_common(100)

[('的', 184244),
 ('在', 47370),
 ('了', 36722),
 ('和', 30809),
 ('是', 30283),
 ('月', 18711),
 ('也', 15995),
 ('年', 15971),
 ('有', 14714),
 ('为', 14448),
 ('等', 14340),
 ('将', 14060),
 ('对', 13074),
 ('与', 12568),
 ('日', 12322),
 ('中', 11117),
 ('中国', 11036),
 ('6', 10477),
 ('上', 10192),
 ('不', 10027),
 ('他', 9530),
 ('都', 9447),
 ('发展', 8795),
 ('企业', 8584),
 ('就', 8537),
 ('到', 8338),
 ('市场', 8095),
 ('但', 7729),
 ('这', 7658),
 ('被', 7575),
 ('从', 7513),
 ('并', 7412),
 ('人', 7339),
 ('后', 7084),
 ('公司', 6915),
 ('一个', 6772),
 ('说', 6703),
 ('新', 6467),
 ('表示', 6309),
 ('要', 6276),
 ('还', 6245),
 ('会', 6179),
 ('个', 6176),
 ('我', 6141),
 ('而', 6090),
 ('进行', 5802),
 ('我们', 5742),
 ('记者', 5734),
 ('以', 5615),
 ('5', 5569),
 ('工作', 5135),
 ('没有', 5000),
 ('美国', 4840),
 ('下', 4741),
 ('更', 4739),
 ('通过', 4720),
 ('大', 4704),
 ('让', 4701),
 ('可以', 4681),
 ('经济', 4670),
 ('时', 4654),
 ('目前', 4645),
 ('国家', 4628),
 ('项目', 4538),
 ('问题', 4422),
 ('创新', 4416),
 ('多', 4410),
 ('已经', 4391),
 ('建设

In [25]:
def prob_1(word): return words_count[word] / len(token)

In [26]:
prob_1('我们')

0.0015587203508559526

In [27]:
token[:10]

['此外', '自', '本周', '6', '月', '12', '日起', '除', '小米', '手机']

In [28]:
token_2_gram = [token[i]+token[i+1] for i in range(len(token)-1)]

In [29]:
token_2_gram[:10]

['此外自', '自本周', '本周6', '6月', '月12', '12日起', '日起除', '除小米', '小米手机', '手机6']

In [30]:
words_count_2 = Counter(token_2_gram)
words_count_2.most_common(10)

[('6月', 8502),
 ('2016年', 2773),
 ('2017年', 2496),
 ('的是', 2250),
 ('5月', 2111),
 ('也是', 2054),
 ('nannan', 1740),
 ('都是', 1590),
 ('自己的', 1496),
 ('更多', 1490)]

In [31]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: 
        return words_count_2[word1+word2] / len(token_2_gram)
    else:
        return 1 / len(token_2_gram)

In [32]:
prob_2('在', '吃饭')

2.7145955659796026e-07

In [33]:
prob_2('我们', '在')

4.696250329144712e-05

In [34]:
def get_probability(sentence):
    words = cut(sentence)
    
    p = 1
    for i in range(len(words)-1):
        p *= prob_2(words[i], words[i+1])
    return p

In [35]:
get_probability('小明今天抽奖抽到一台苹果手机')

6.402517805790038e-39

In [36]:
get_probability('小明今天抽奖抽到一架波音飞机')

4.001573628618774e-40

In [37]:
get_probability('洋葱奶昔来一杯')

2.000393368470005e-20

In [38]:
for sen in [generate(gram=example_grammar, target='sentence') for _ in range(10)]:
    print('sentence: {} with prob: {}'.format(sen, get_probability(sen)))

sentence: 一个女人听着一个小小的女人 with prob: 2.6506423715970752e-36
sentence: 这个小小的好看的蓝色的桌子听着一个小小的小小的小猫 with prob: 1.0267324913463742e-72
sentence: 一个小小的小猫听着一个小猫 with prob: 6.626605928992687e-37
sentence: 这个好看的小小的小小的蓝色的桌子听着这个好看的蓝色的篮球 with prob: 8.251194926082362e-91
sentence: 一个桌子坐在一个小猫 with prob: 5.4302589682636776e-27
sentence: 这个女人坐在这个好看的小小的小猫 with prob: 2.3897838864216748e-45
sentence: 这个小小的好看的小猫看见这个蓝色的小猫 with prob: 7.044154788117434e-58
sentence: 一个女人看着这个女人 with prob: 8.688414349221884e-26
sentence: 一个好看的好看的好看的篮球看着这个女人 with prob: 1.0411946072197226e-61
sentence: 这个蓝色的小猫看见一个蓝色的蓝色的桌子 with prob: 9.790480053195557e-62


In [39]:
need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]
for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = map(get_probability, [s1, s2])
    
    better = s1 if p1 > p2 else s2
    print('{} is more possible'.format(better))
    print('-'*4+'{} with probability {}'.format(s1, p1))
    print('-'*4+'{} with probability {}'.format(s2, p2))

明天晚上请你吃大餐，我们一起吃苹果 is more possible
----今天晚上请你吃大餐，我们一起吃日料 with probability 1.7522734186117049e-62
----明天晚上请你吃大餐，我们一起吃苹果 with probability 1.7522734186117049e-62
真是一只好看的小猫 is more possible
----真事一只好看的小猫 with probability 1.6215052609106503e-32
----真是一只好看的小猫 with probability 5.973284865090045e-26
今晚我去吃火锅 is more possible
----今晚我去吃火锅 with probability 8.40165214757402e-19
----今晚火锅去吃我 with probability 1.9548932285749235e-25
养乐多绿来一杯 is more possible
----洋葱奶昔来一杯 with probability 2.000393368470005e-20
----养乐多绿来一杯 with probability 7.36902908683612e-14


### 问答和编程练习

#### 基础理论部分

#### 编程实践部分

##### 设计自己的句子生成器

In [40]:
def generate_n(grammar, target, n):
    res = []
    for _ in range(n):
        res.append(generate(grammar, target))
    return res

In [41]:
singer = """
singer = 姓名 程度副词* 擅长 特点
姓名 = 华晨宇 | 毛不易
程度副词* = null | 程度副词 程度副词*
程度副词 = 很 | 非常 | 特别 | 极其
擅长 = 喜欢 | 善于 | 擅长
特点 = 作词 | 作曲 | 喝酒 | 怼人
"""

In [42]:
singer_grammar = create_grammar(singer, split_='=')
singer_grammar

{'singer': [['姓名', '程度副词*', '擅长', '特点']],
 '姓名': [['华晨宇'], ['毛不易']],
 '程度副词*': [['null'], ['程度副词', '程度副词*']],
 '程度副词': [['很'], ['非常'], ['特别'], ['极其']],
 '擅长': [['喜欢'], ['善于'], ['擅长']],
 '特点': [['作词'], ['作曲'], ['喝酒'], ['怼人']]}

In [43]:
generate(singer_grammar, 'singer')

'华晨宇善于作曲'

In [44]:
generate_n(singer_grammar, 'singer', 6)

['毛不易喜欢怼人', '毛不易善于作词', '毛不易极其喜欢作词', '毛不易善于作词', '华晨宇喜欢喝酒', '华晨宇非常喜欢怼人']

In [45]:
self_introduction = """
self_introduction = 礼貌称呼 报姓名 来处 主修 喜欢 具体爱好*
礼貌称呼 = null | 您好， | 你们好， | 老师们好，
报姓名 = 我 动词 姓名，
动词 = 是 | 叫
姓名 = xhp | bxf
来处 = 来自 学校，
学校 = 同济大学 | 华东师范大学 | 中南大学
主修 = null | 专业是 专业，
专业 = 植物学 | 采矿工程 | 土木工程
喜欢 = 喜欢 | 擅长 | 善于
具体爱好* = 具体爱好 具体爱好*
"""

In [46]:
self_introduction_grammar = create_grammar(self_introduction, split_='=')
self_introduction_grammar

{'self_introduction': [['礼貌称呼', '报姓名', '来处', '主修', '喜欢', '具体爱好*']],
 '礼貌称呼': [['null'], ['您好，'], ['你们好，'], ['老师们好，']],
 '报姓名': [['我', '动词', '姓名，']],
 '动词': [['是'], ['叫']],
 '姓名': [['xhp'], ['bxf']],
 '来处': [['来自', '学校，']],
 '学校': [['同济大学'], ['华东师范大学'], ['中南大学']],
 '主修': [['null'], ['专业是', '专业，']],
 '专业': [['植物学'], ['采矿工程'], ['土木工程']],
 '喜欢': [['喜欢'], ['擅长'], ['善于']],
 '具体爱好*': [['具体爱好', '具体爱好*']]}

In [1]:
generate(self_introduction_grammar, 'self_introduction')

NameError: name 'generate' is not defined